<a href="https://colab.research.google.com/github/990921HVQ/Prote-nas-para-acoplamiento-molecular/blob/main/2_3_Estructuras_para_docking_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biotite
!pip install redo
!pip install urllib3
!pip install requests_cache
!pip install nglview
!pip install pypdb
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.8 MB 64 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for biotite: filename=biotite-0.35.0-cp37-cp37m-linux_x86_64.whl size=38774098 sha256=2b5ff0d2404416dbc3b4563ae660f0fb1bb50a381c41f7933982748272195762
  Stored in directory: /root/.cache/pip/wheels/46/8e/c5/3f067da57892eae88984d418d44b8212be171953891058ab37
Successfully built biotite
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for redo: filename=redo-2.0.4-py2.py3-none-any.whl size=11947 sha256=ae6eed1339548d546d0b73916a8ba598555c87641eff09ffb5eaae6f6a6d0b72
  Stored in directory: /root/.cache/pip/wh

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import biotite
import biotite.database.rcsb as rcsb
from datetime import datetime, time

import collections
import logging
import pathlib
import time
import warnings
import datetime

import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import redo
import requests_cache
import nglview
import pypdb
import biotite.database.rcsb as rcsb
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

In [ ]:
Dianas2  = {'Diana epigenética': ['DNMT1', 'DNMT3a', 'DNMT3b', 'Ep300', 'EZH2', 'HDAC1', 'HDAC2', 'HDAC4', 'HDAC5', 'HDAC7', 'HDAC8', 'HDAC9', 'HDAC10', 'HDAC11', 'LSD1', 'SIRT1', 'SIRT3', 'KDM1B', 'KDM6A', 'BRD4'],
           'Herramienta epigenética': ['Escritor', 'Escritor', 'Escritor', 'Escritor', 'Escritor', 'Borrador', 'Borrador', 'Borrador',  'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Borrador', 'Lector'],
            'UniProt ID': ['P26358', 'Q9Y6K1', 'Q9UBC3', 'Q09472', 'Q15910', 'Q13547', 'Q92769', 'P56524',  'Q9UQL6', 'Q8WUI4', 'Q9BY41', 'Q9UKV0', 'Q969S8', 'Q96DB2', 'O60341', 'Q96EB6', 'Q9NTG7', 'Q8NB78', 'O15550', 'O60885'],
            'Número total de estructuras en 2022': ['19', '18', '26', '26', '26', '8', '27', '17', '1', '5', '53', '0', '0', '0', '89', '8', '31', '12', '5', '477'],
}

Dianas2 = pd.DataFrame (Dianas2, columns= ['Diana epigenética','Herramienta epigenética', 'UniProt ID', 'Número total de estructuras en 2022'])

Dianas2

,Diana epigenética,Herramienta epigenética,UniProt ID,Número total de estructuras en 2022
0,DNMT1,Escritor,P26358,19
1,DNMT3a,Escritor,Q9Y6K1,18
2,DNMT3b,Escritor,Q9UBC3,26
3,Ep300,Escritor,Q09472,26
4,EZH2,Escritor,Q15910,26
5,HDAC1,Borrador,Q13547,8
6,HDAC2,Borrador,Q92769,27
7,HDAC4,Borrador,P56524,17
8,HDAC5,Borrador,Q9UQL6,1
9,HDAC7,Borrador,Q8WUI4,5


## DNMT1

In [ ]:
uniprot_id= "P26358"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", less=before_deposition_date
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_deposition_date,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 6
Selected PDB IDs:
4Z97 4Z96 4WXX 6L1F 3EPZ 4YOC


## DNMT3a

In [ ]:
uniprot_id= "Q9Y6K1"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", less=before_deposition_date
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 3
Selected PDB IDs:
8BA5 3A1B 3A1A


## DNMT3b

In [ ]:
uniprot_id= "Q9UBC3"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", less=before_deposition_date
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_deposition_date,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 9
Selected PDB IDs:
5NV2 5NV0 5NVO 5NRV 6R3E 5NRR 5NV7 5NR3 5NRS


## Ep300

In [ ]:
uniprot_id= "Q09472"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 9
Selected PDB IDs:
7VI0 7QGS 6PGU 6PF1 7VHZ 5LPM 7VHY 4BHW 5NU5


## EZH2

In [ ]:
uniprot_id= "Q15910"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 2
Selected PDB IDs:
4MI5 4MI0


## HDAC1

In [ ]:
uniprot_id= "Q13547"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 2
Selected PDB IDs:
7SME 4BKX


## HDAC2

In [ ]:
uniprot_id= "Q92769"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 3
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 30
Selected PDB IDs:
5IWG 4LY1 6G3O 5IX0 7MOY 7MOX 7MOS 4LXZ 8A0B 7MOT 7LTG 7MOZ 7LTL 6XDM 7LTK 7ZZO 7JS8 6WBW 3MAX 7ZZU 6XEB 7ZZT 6XEC 7ZZW 7KBH 7ZZP 7KBG 7ZZS 6WBZ 7ZZR


## HDAC4

In [ ]:
uniprot_id= "P56524"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 8
Selected PDB IDs:
5ZOO 5ZOP 2VQQ 2VQO 3V31 3UXG 5A2S 3UZD


## HDAC5

In [ ]:
uniprot_id= "Q9UQL6"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 4
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 1
Selected PDB IDs:
5UWI


## HDAC7

In [ ]:
uniprot_id= "Q8WUI4"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 3
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 5
Selected PDB IDs:
3C10 3ZNS 3ZNR 3C0Z 3C0Y


## HDAC8

In [ ]:
uniprot_id= "Q9BY41"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 8
Selected PDB IDs:
3MZ7 3MZ6 3EW8 3SFH 3SFF 4QA4 1T67 1T69


## HDAC9

In [ ]:
uniprot_id= "Q9UKV0"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 0
Selected PDB IDs:



## HDAC10

In [ ]:
uniprot_id= "Q969S8"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 0
Selected PDB IDs:



## HDAC11

In [ ]:
uniprot_id= "Q96DB2"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 0
Selected PDB IDs:



## LSD1

In [ ]:
uniprot_id= "O60341"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 19
Selected PDB IDs:
2Z5U 7E0G 7XW8 7VQT 7VQS 2DW4 7VQU 2Z3Y 2HKO 2EJR 6NQM 6KGQ 2H94 6KGR 6KGO 6KGP 6NR5 7W3L 6NQU


## SIRT1

In [ ]:
uniprot_id= "Q96EB6"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 4
Selected PDB IDs:
4ZZJ 4IF6 4KXQ 4I5I


## SIRT3

In [ ]:
uniprot_id= "Q9NTG7"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 7
Selected PDB IDs:
4JSR 4BN4 4O8Z 4BV3 4JT9 4JT8 4BVB


## KDM1B

In [ ]:
uniprot_id= "Q8NB78"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 2
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 8
Selected PDB IDs:
4FWJ 4GUU 4GUT 4FWF 7XE2 7XE1 4GU1 7XE3


## KDM6A

In [ ]:
uniprot_id= "O15550"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 4
Selected PDB IDs:
6FUL 6FUK 6G8F 3AVS


## BRD4

In [ ]:
uniprot_id= "O60885"
before_deposition_date = "2022-09-01T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"Number of matches: {len(pdb_ids)}")
print("Selected PDB IDs:")
print(*pdb_ids)

Number of matches: 385
Selected PDB IDs:
5Z1S 5Z1T 6SAJ 5Z1R 6SAH 4LZR 4GPJ 5WMA 6SB8 4LZS 4ZW1 5WMG 5WMD 5EIS 7MRB 4LYI 7MRA 7A9U 7JKZ 7JKY 6SA3 6SA2 5YOV 5YOU 7KO0 4LYW 6IN1 5KU3 4LYS 5EI4 4QB3 6FFD 4J3I 5LUU 4CFL 4CFK 4UIZ 4UIY 7MR9 7JKX 7JKW 7P6W 7P6V 7MR8 7M16 7R9C 7AXR 5UET 5UER 5UES 5UEP 5UEQ 5UF0 5UEZ 5HM0 5UEX 5UEY 5UEV 5HLS 6YIN 5O97 5M3A 7RXS 4J0R 4IOR 4IOQ 7RXT 5LRQ 4IOO 4J0S 7ZG2 4WIV 7ZG1 5OWM 7ZFS 5XI3 5XI4 6JJB 5XI2 7ZFO 7ZFN 7ZFY 7ZFZ 7ZFU 7ZFT 5XHY 7ZFV 7O18 3MXF 7ZEF 7Q3F 7ZEN 4WHW 6JJ5 7MLS 6JJ6 6JJ3 7MLR 7MLQ 7C6P 5FBX 6CZU 4O7C 4O7B 4O7A 7RUH 7RUI 5VBO 4LRG 6CZV 5N2M 7ZE6 6JI4 6HDQ 6JI5 6JI3 4NUC 7LH8 6RWJ 6PSB 5A85 4NUE 4NUD 4O72 7UZN 4O78 4LR6 4O75 6C7Q 6C7R 6ZED 6ZEL 6ZF9 5IGK 4QZS 6PS9 6PRT 7ZAD 7ZAE 7ZAA 7B1T 7ZAJ 5A5S 7ZAT 6SWQ 5S9R 6LIH 6SWN 7ZAR 7ZAQ 6S6K 4A9L 4MR4 4MR3 6LIM 4YH3 6ZCI 7ZA6 5WA5 7C2Z 4YH4 6P05 7ZA7 5KJ0 5HCL 7ZA9 6LG8 5MKZ 6LG9 6LG6 6LG7 6LG4 6LG5 7T3F 5CY9 6S4B 6KEG 6KEH 6KEE 5MLI 4NQM 6KEF 6KEC 6KED 4NR8 6KEK 6KEI 6KEJ 7UTY 6ZB3 5LJ2 5LJ1